### Membuat 5 Dataset Hadist 

In [1]:
import numpy as np
import pandas as pd
import spacy
from deep_translator import GoogleTranslator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def translate_to_english(text):
    translated_text = GoogleTranslator(source='indonesian', target='english').translate(text)
    return translated_text

def translate_to_indonesian(text):
    translated_text = GoogleTranslator(source='english', target='indonesian').translate(text)
    return translated_text

def find_name_with_spacy(paragraph):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(paragraph)
    names = [entity.text for entity in doc.ents if entity.label_ == 'PERSON']
    return names

c:\Users\Tirta\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
musnad_syafii = pd.read_csv("D:/RoadMap Kuliah/Semester 7/Quran Hadis Mining/tugasku/dataset hadist/musnad_syafii.csv",sep=";")
muwatho_malik = pd.read_csv("D:/RoadMap Kuliah/Semester 7/Quran Hadis Mining/tugasku/dataset hadist/muwatho_malik.csv",sep=";")
shahih_bukhari = pd.read_csv("D:/RoadMap Kuliah/Semester 7/Quran Hadis Mining/tugasku/dataset hadist/shahih_bukhari.csv",sep=";")
shahih_muslim = pd.read_csv("D:/RoadMap Kuliah/Semester 7/Quran Hadis Mining/tugasku/dataset hadist/shahih_muslim.csv",sep=";")

In [3]:
for i in range(len(musnad_syafii)):
    musnad_syafii.loc[i, "terjemah"] = musnad_syafii.loc[i, "terjemah"].replace(f"Musnad Syafi'i {i+1}: ", "")

In [4]:
nama_atribut = ["id","kitab","arab","terjemah"]
df_hadisku = pd.DataFrame(columns=nama_atribut)
df_hadisku

,id,kitab,arab,terjemah


### Pengambilan 2 hadist dari musnad syafii

In [5]:
hadist_syafii = musnad_syafii[musnad_syafii['id'].isin([50, 85])][nama_atribut]
df_hadisku = pd.concat([df_hadisku, hadist_syafii])

In [6]:
df_hadisku

,id,kitab,arab,terjemah
49,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...
84,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...


### Pengambilan 1 hadist dari tiap tiap hadist yang belum diambil

In [7]:
hadist_malik = muwatho_malik[muwatho_malik['id'].isin([964])][nama_atribut]
hadist_bukhari = shahih_bukhari[shahih_bukhari['id'].isin([1077])][nama_atribut]
hadist_muslim = shahih_muslim[shahih_muslim['id'].isin([4])][nama_atribut]
df_hadisku = pd.concat([df_hadisku,hadist_malik, hadist_bukhari, hadist_muslim])

In [8]:
df_hadisku

,id,kitab,arab,terjemah
49,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...
84,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...
963,964,muwatho_malik,حَدَّثَنِي يَحْيَى عَنْ مَالِك عَنْ مُحَمَّدِ ...,Telah menceritakan kepadaku Yahya dari Malik d...
1076,1077,shahih_bukhari,حَدَّثَنَا عَبْدُ اللَّهِ بْنُ مَسْلَمَةَ عَنْ...,Telah menceritakan kepada kami ['Abdullah bin ...
3,4,shahih_muslim,و حَدَّثَنَا مُحَمَّدُ بْنُ عُبَيْدٍ الْغُبَرِ...,Telah menceritakan kepada kami [Muhammad bin U...


### 5 Dataset Yang Akan dipergunakan

In [9]:
df_hadisku = df_hadisku.reset_index(drop=True)
df_hadisku

,id,kitab,arab,terjemah
0,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...
1,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...
2,964,muwatho_malik,حَدَّثَنِي يَحْيَى عَنْ مَالِك عَنْ مُحَمَّدِ ...,Telah menceritakan kepadaku Yahya dari Malik d...
3,1077,shahih_bukhari,حَدَّثَنَا عَبْدُ اللَّهِ بْنُ مَسْلَمَةَ عَنْ...,Telah menceritakan kepada kami ['Abdullah bin ...
4,4,shahih_muslim,و حَدَّثَنَا مُحَمَّدُ بْنُ عُبَيْدٍ الْغُبَرِ...,Telah menceritakan kepada kami [Muhammad bin U...


### Menambahkan Sanat dan Matan Pada Dataset Yang telah dibuat

In [10]:
df_hadisku['english'] = df_hadisku['terjemah'].apply(translate_to_english)

In [11]:
hasil_perawi = []
hasil_matan_inggris = []

kata_kunci = [
    '\r\n\r\n', 'say :', ': That', 'said,', 'stated:', 'say:', 'said:',
    '. That', ', ""', ';', ', "', 'bersabda', 'about',
    "sallallaahu 'alaihi wa sallam", "he said", "Rasulullah",
    "from the Prophet", "with a similar editorial", "with the same",
    "with similar", "namely a similar hadith", "with a hadith similar",
    "with an editorial similar","Umar bin Al Khaththab wrote a letter",":", "such as", "that", "regarding"
]

for i in range(len(df_hadisku)):
    matan = ""
    terjemah = df_hadisku.loc[i, "english"]
    perawi = "program gagal"
    for kunci in kata_kunci:
        if kunci in terjemah:
            kalimat_terpisah = terjemah.split(kunci)
            perawi = find_name_with_spacy(kalimat_terpisah[0])
            matan = kunci.join(kalimat_terpisah[1:])
            break
    
    if matan == "":
        matan = "program gagal"
        
    hasil_perawi.append(perawi)
    hasil_matan_inggris.append(matan)

In [12]:
hasil_perawi

[['Abdul Aziz bin Muhammad', 'Zaid bin Aslam', 'Atha bin Yasar', 'Ibn Abbas'],
 ['Abdul Majid bin Abdul Aziz',
  'Ibn Juraij',
  'Abu Az-Zubair',
  'Jabir bin Abdullah'],
 ['Yahya',
  'Malik',
  'Muhammad bin Yahya bin Habban',
  "Al A'raj",
  'Abu Hurairah',
  "the Messenger of Allah salla Allahu '"],
 ['Abdullah bin Maslamah',
  'Malik',
  'Ibn Shihab',
  'Abu Salamah',
  'Abdullah Al Aghor',
  'Abu Hurairah'],
 ['Muhammad bin Ubaid al-Ghubari',
  'Abu Awanah',
  'Abu Hashin',
  'Abu Salih',
  'Abu Hurairah']]

In [13]:
def menghilangkan_list_tertentu(lst):
    kata_kunci = ["rasulullah", "prophet", "the Messenger of Allah"]
    hasil = []
    for sublst in lst:
        sementara = [x for x in sublst if all(keyword.lower() not in x.lower() for keyword in kata_kunci)]
        hasil.append(sementara)
    return hasil

hasil_perawi = menghilangkan_list_tertentu(hasil_perawi)
hasil_perawi

[['Abdul Aziz bin Muhammad', 'Zaid bin Aslam', 'Atha bin Yasar', 'Ibn Abbas'],
 ['Abdul Majid bin Abdul Aziz',
  'Ibn Juraij',
  'Abu Az-Zubair',
  'Jabir bin Abdullah'],
 ['Yahya',
  'Malik',
  'Muhammad bin Yahya bin Habban',
  "Al A'raj",
  'Abu Hurairah'],
 ['Abdullah bin Maslamah',
  'Malik',
  'Ibn Shihab',
  'Abu Salamah',
  'Abdullah Al Aghor',
  'Abu Hurairah'],
 ['Muhammad bin Ubaid al-Ghubari',
  'Abu Awanah',
  'Abu Hashin',
  'Abu Salih',
  'Abu Hurairah']]

In [14]:
hasil_matan_inggris

[' "The Messenger of Allah performed ablution, for this he put his hands into a container, then did istinshaq and rinsed his mouth at the same time. Then he put his hand in and poured the water over his face in one stroke, then poured the water over both his hands once, then wiped his head and both ears in one stroke.” 57',
 'I once saw Rasulullah SAW performing sunnah prayers on his vehicle facing all directions. 92',
 ' "Let none of you propose to his brother."',
 ' "Rabb Our Tabaaraka wa Ta\'ala descends every night into the heavens of the world in the last third of the night and says: "Whoever prays to Me I will surely grant and whoever asks Me I will definitely fulfill and whoever asks Me for forgiveness I will definitely forgive."',
 ' "The Messenger of Allah sallallaahu \'alaihi wasallam said: "Whoever lies about my name, let him take his seat from hell."']

In [15]:
hasil_matan = [translate_to_indonesian(kalimat) for kalimat in hasil_matan_inggris]
hasil_matan 

['“Rasulullah berwudhu, untuk itu beliau memasukkan tangannya ke dalam sebuah wadah, lalu beristinshaq dan sekaligus berkumur. Kemudian beliau memasukkan tangannya ke dalam dan menuangkan air tersebut ke wajahnya dalam sekali usapan, lalu menuangkannya siramlah kedua tangannya satu kali, lalu usaplah kepala dan kedua telinganya dengan sekali usap.” 57',
 'Saya pernah melihat Rasulullah SAW menunaikan shalat sunnah di atas kendaraannya menghadap ke segala arah. 92',
 '“Janganlah ada di antara kalian yang melamar saudaranya.”',
 '“Rabb Tabaaraka wa Ta’ala kami turun setiap malam ke langit dunia pada sepertiga malam terakhir dan bersabda: “Barangsiapa yang berdoa kepada-Ku pasti akan Aku kabulkan dan siapa yang meminta kepada-Ku pasti akan Kukabulkan dan barangsiapa yang meminta ampun kepada-Ku. Aku pasti akan memaafkannya."',
 '“Rasulullah Shallallahu ‘alaihi wasallam bersabda: “Barangsiapa berbohong tentang namaku, hendaklah dia mengambil tempat duduknya dari neraka.”']

In [16]:
df_hadisku["Perawi"] = hasil_perawi
df_hadisku["Matan"] = hasil_matan

In [17]:
df_hadisku

,id,kitab,arab,terjemah,english,Perawi,Matan
0,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...,Abdul Aziz bin Muhammad reported to us from Za...,"[Abdul Aziz bin Muhammad, Zaid bin Aslam, Atha...","“Rasulullah berwudhu, untuk itu beliau memasuk..."
1,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...,Abdul Majid bin Abdul Aziz reported to us from...,"[Abdul Majid bin Abdul Aziz, Ibn Juraij, Abu A...",Saya pernah melihat Rasulullah SAW menunaikan ...
2,964,muwatho_malik,حَدَّثَنِي يَحْيَى عَنْ مَالِك عَنْ مُحَمَّدِ ...,Telah menceritakan kepadaku Yahya dari Malik d...,Yahya has told me from Malik from [Muhammad bi...,"[Yahya, Malik, Muhammad bin Yahya bin Habban, ...",“Janganlah ada di antara kalian yang melamar s...
3,1077,shahih_bukhari,حَدَّثَنَا عَبْدُ اللَّهِ بْنُ مَسْلَمَةَ عَنْ...,Telah menceritakan kepada kami ['Abdullah bin ...,It has been told to us ['Abdullah bin Maslamah...,"[Abdullah bin Maslamah, Malik, Ibn Shihab, Abu...",“Rabb Tabaaraka wa Ta’ala kami turun setiap ma...
4,4,shahih_muslim,و حَدَّثَنَا مُحَمَّدُ بْنُ عُبَيْدٍ الْغُبَرِ...,Telah menceritakan kepada kami [Muhammad bin U...,Has told us [Muhammad bin Ubaid al-Ghubari] ha...,"[Muhammad bin Ubaid al-Ghubari, Abu Awanah, Ab...",“Rasulullah Shallallahu ‘alaihi wasallam bersa...


In [18]:
df_hadisku.to_csv("D:/RoadMap Kuliah/Semester 7/Quran Hadis Mining/tugasku/dataset hadist/df_hadisku.csv", index = False)

### Pencocokan Hadist dengan Tema dan Ayat Alquran 

In [19]:
file_path = "D:\RoadMap Kuliah\Semester 7\Quran Hadis Mining\Dataset-al-quran\dataset index Al Qur'an.xlsx"
sheet_name1 = 'BAB'
sheet_name2 = 'No Duplikat'
df_tema = pd.read_excel(file_path, sheet_name=sheet_name1)
df_temaayat = pd.read_excel(file_path, sheet_name=sheet_name2)
data_terjemah = pd.read_csv("D:\RoadMap Kuliah\Semester 7\Quran Hadis Mining\Dataset-al-quran\DataSet-Terjemah Al Quran.csv",sep=";")
data_terjemah['id'] = data_terjemah['id'].str.replace('.', ':')
data_arab = pd.read_csv("D:\RoadMap Kuliah\Semester 7\Quran Hadis Mining\Dataset-al-quran\Arabic-Original.csv", sep="|")

In [20]:
def pencarian_tema(text):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(data_terjemah['terjemah'].values.tolist() + [text])

    cosine_similarities = cosine_similarity(tfidf_matrix[:-1], tfidf_matrix[-1])
    
    most_similar_indices = np.argsort(cosine_similarities, axis=0)[-3:].flatten()[::-1]
    persentage_most_similar_indices = cosine_similarities[most_similar_indices] * 100

    for i, index in enumerate(most_similar_indices):
        target = data_terjemah.loc[index, 'id']
        result = df_temaayat[df_temaayat['ayat'].apply(lambda x: x == target)]
        result = result['bab'].tolist()
        print(f"Ranking ke-{i + 1}")
        print(f"Hadist Yang anda inputkan :\n{text}\n===============================\nTeridentifikasi Menjadi {len(result)} Tema yaitu")
        for j in range(len(result)):
            print(j+1,df_tema[df_tema['ID'] == result[j]]['Deskripsi'].values[0])
        
        print(f"Urutan ke-{i+1} Teridentifikasi Isinya Paling Cenderung Mendekati Pada")
        print(f"Surah ke-{data_arab['Surat'][index]} dan Nomor Ayat ke-{data_arab['NAyat'][index]}")
        print(f"Memiliki Persentase Kemiripan {persentage_most_similar_indices[i]} %")
        print("Berikut Ayatnya :")
        print(data_arab['Ayat'][index])
        print(f"Berikut Terjemahan dari Ayat tersebut : {data_terjemah['terjemah'][index]}")
        print("\n")


for index, row in df_hadisku.iterrows():
    text = row['Matan']
    print(f"===== Analisis untuk baris ke-{index + 1} =====")
    pencarian_tema(text)
    print("\n")

===== Analisis untuk baris ke-1 =====
Ranking ke-1
Hadist Yang anda inputkan :
“Rasulullah berwudhu, untuk itu beliau memasukkan tangannya ke dalam sebuah wadah, lalu beristinshaq dan sekaligus berkumur. Kemudian beliau memasukkan tangannya ke dalam dan menuangkan air tersebut ke wajahnya dalam sekali usapan, lalu menuangkannya siramlah kedua tangannya satu kali, lalu usaplah kepala dan kedua telinganya dengan sekali usap.” 57
Teridentifikasi Menjadi 2 Tema yaitu
1 Arkanul Islam
2 Iman
Urutan ke-1 Teridentifikasi Isinya Paling Cenderung Mendekati Pada
Surah ke-69 dan Nomor Ayat ke-30
Memiliki Persentase Kemiripan [23.95271537] %
Berikut Ayatnya :
خُذُوهُ فَغُلُّوهُ
Berikut Terjemahan dari Ayat tersebut : (Allah berfirman): Peganglah dia lalu belenggulah tangannya ke lehernya.


Ranking ke-2
Hadist Yang anda inputkan :
“Rasulullah berwudhu, untuk itu beliau memasukkan tangannya ke dalam sebuah wadah, lalu beristinshaq dan sekaligus berkumur. Kemudian beliau memasukkan tangannya ke dalam

In [22]:
def pencarian_tema2(text):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(data_terjemah['terjemah'].values.tolist() + [text])

    cosine_similarities = cosine_similarity(tfidf_matrix[:-1], tfidf_matrix[-1])
    
    most_similar_indices = np.argsort(cosine_similarities, axis=0)[-3:].flatten()[::-1]
    persentage_most_similar_indices = cosine_similarities[most_similar_indices] * 100

    hasil_pencarian = []
    for i, index in enumerate(most_similar_indices):
        target = data_terjemah.loc[index, 'id']
        result = df_temaayat[df_temaayat['ayat'].apply(lambda x: x == target)]
        result = result['bab'].tolist()
        for j in range(len(result)):
            top_tema = i+1
            tema = df_tema[df_tema['ID'] == result[j]]['Deskripsi'].values[0]
            surahNomorAyat = f"{data_arab['Surat'][index]}:{data_arab['NAyat'][index]}"
            ayat = data_arab['Ayat'][index]
            terjemah = data_terjemah['terjemah'][index]
            persentase = persentage_most_similar_indices[i]
            hasil_pencarian.append({'top_tema': top_tema, 'tema': tema, 'surahNomorAyat': surahNomorAyat, 'ayatQuran': ayat, 'terjemahQuran': terjemah, 'persentase': "{:.2f}%".format(float(persentase))})
    return hasil_pencarian

df_hasil_hadist = pd.DataFrame(columns=['id', 'kitab', 'arab', 'terjemah', 'english', 'Perawi', 'Matan', 'top_tema', 'tema', 'surahNomorAyat', 'ayatQuran', 'terjemahQuran', 'persentase'])
for i, row in df_hadisku.iterrows():
    hasil_pencarian = pencarian_tema2(row['Matan'])
    for j, hasil in enumerate(hasil_pencarian):
        df_hasil_hadist = pd.concat([df_hasil_hadist, pd.DataFrame([{'id': row['id'], 'kitab': row['kitab'], 'arab': row['arab'], 'terjemah': row['terjemah'], 'english': row['english'], 'Perawi': row['Perawi'], 'Matan': row['Matan'], 'top_tema': hasil['top_tema'], 'tema': hasil['tema'], 'surahNomorAyat': hasil['surahNomorAyat'], 'ayatQuran': hasil['ayatQuran'], 'terjemahQuran': hasil['terjemahQuran'], 'persentase': hasil['persentase']}])], ignore_index=True)

In [23]:
df_hasil_hadist

,id,kitab,arab,terjemah,english,Perawi,Matan,top_tema,tema,surahNomorAyat,ayatQuran,terjemahQuran,persentase
0,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...,Abdul Aziz bin Muhammad reported to us from Za...,"[Abdul Aziz bin Muhammad, Zaid bin Aslam, Atha...","“Rasulullah berwudhu, untuk itu beliau memasuk...",1,Arkanul Islam,69:30,خُذُوهُ فَغُلُّوهُ,(Allah berfirman): Peganglah dia lalu belenggu...,23.95%
1,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...,Abdul Aziz bin Muhammad reported to us from Za...,"[Abdul Aziz bin Muhammad, Zaid bin Aslam, Atha...","“Rasulullah berwudhu, untuk itu beliau memasuk...",1,Iman,69:30,خُذُوهُ فَغُلُّوهُ,(Allah berfirman): Peganglah dia lalu belenggu...,23.95%
2,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...,Abdul Aziz bin Muhammad reported to us from Za...,"[Abdul Aziz bin Muhammad, Zaid bin Aslam, Atha...","“Rasulullah berwudhu, untuk itu beliau memasuk...",2,Arkanul Islam,57:6,يُولِجُ اللَّيْلَ فِي النَّهَارِ وَيُولِجُ الن...,Dialah yang memasukkan malam ke dalam siang da...,23.22%
3,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...,Abdul Aziz bin Muhammad reported to us from Za...,"[Abdul Aziz bin Muhammad, Zaid bin Aslam, Atha...","“Rasulullah berwudhu, untuk itu beliau memasuk...",2,Cabang-cabang Ilmu,57:6,يُولِجُ اللَّيْلَ فِي النَّهَارِ وَيُولِجُ الن...,Dialah yang memasukkan malam ke dalam siang da...,23.22%
4,50,musnad_syafii,مسند الشافعي 50: أَخْبَرَنَا عَبْدُ الْعَزِيزِ...,Abdul Aziz bin Muhammad mengabarkan kepada kam...,Abdul Aziz bin Muhammad reported to us from Za...,"[Abdul Aziz bin Muhammad, Zaid bin Aslam, Atha...","“Rasulullah berwudhu, untuk itu beliau memasuk...",3,Arkanul Islam,74:42,مَا سَلَكَكُمْ فِي سَقَرَ,Apakah yang memasukkan kamu ke dalam Saqar (ne...,22.38%
5,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...,Abdul Majid bin Abdul Aziz reported to us from...,"[Abdul Majid bin Abdul Aziz, Ibn Juraij, Abu A...",Saya pernah melihat Rasulullah SAW menunaikan ...,1,Arkanul Islam,8:48,وَإِذْ زَيَّنَ لَهُمُ الشَّيْطَانُ أَعْمَالَهُ...,Dan ketika setan menjadikan mereka memandang b...,18.74%
6,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...,Abdul Majid bin Abdul Aziz reported to us from...,"[Abdul Majid bin Abdul Aziz, Ibn Juraij, Abu A...",Saya pernah melihat Rasulullah SAW menunaikan ...,1,Iman,8:48,وَإِذْ زَيَّنَ لَهُمُ الشَّيْطَانُ أَعْمَالَهُ...,Dan ketika setan menjadikan mereka memandang b...,18.74%
7,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...,Abdul Majid bin Abdul Aziz reported to us from...,"[Abdul Majid bin Abdul Aziz, Ibn Juraij, Abu A...",Saya pernah melihat Rasulullah SAW menunaikan ...,1,Jihad,8:48,وَإِذْ زَيَّنَ لَهُمُ الشَّيْطَانُ أَعْمَالَهُ...,Dan ketika setan menjadikan mereka memandang b...,18.74%
8,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...,Abdul Majid bin Abdul Aziz reported to us from...,"[Abdul Majid bin Abdul Aziz, Ibn Juraij, Abu A...",Saya pernah melihat Rasulullah SAW menunaikan ...,1,Akhlak,8:48,وَإِذْ زَيَّنَ لَهُمُ الشَّيْطَانُ أَعْمَالَهُ...,Dan ketika setan menjadikan mereka memandang b...,18.74%
9,85,musnad_syafii,مسند الشافعي 85: أَخْبَرَنَا عَبْدُ الْمَجِيدِ...,Abdul Majid bin Abdul Aziz mengabarkan kepada ...,Abdul Majid bin Abdul Aziz reported to us from...,"[Abdul Majid bin Abdul Aziz, Ibn Juraij, Abu A...",Saya pernah melihat Rasulullah SAW menunaikan ...,2,Arkanul Islam,23:4,وَالَّذِينَ هُمْ لِلزَّكَاةِ فَاعِلُونَ,dan orang-orang yang menunaikan zakat,15.92%


In [24]:
df_hasil_hadist.to_csv("D:/RoadMap Kuliah/Semester 7/Quran Hadis Mining/tugasku/dataset hadist/df_hasil_hadist.csv", index = False)